In [ ]:
import sys
from pathlib import Path

AVES_ROOT = Path("../../..")

EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"
EOD_PATH

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

# esto configura la calidad de la imagen. dependerá de tu resolución. el valor por omisión es 80
mpl.rcParams["figure.dpi"] = 120
# esto depende de las fuentes que tengas instaladas en el sistema.
#mpl.rcParams["font.family"] = "Fira Sans Extra Condensed"


In [ ]:
zones = gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json")
zones.head()

In [ ]:
from aves.data import census

comunas = census.read_census_map('comuna', path=AVES_ROOT / "data" / "external" / "censo_2017_R13")

In [ ]:
comunas_urbanas = comunas[comunas['COMUNA'].isin(zones['Com'].unique())].drop('NOM_COMUNA', axis=1).copy()
comunas_urbanas['NombreComuna'] = comunas_urbanas['COMUNA'].map(dict(zip(zones['Com'], zones['Comuna'])))
comunas_urbanas.plot(facecolor="none", edgecolor="#abacab")

In [ ]:
from aves.features.geo import clip_area_geodataframe
comunas_urbanas = clip_area_geodataframe(comunas_urbanas, zones.total_bounds, buffer=0.05)
comunas_urbanas.plot()

In [ ]:
from aves.models.network import Network


In [ ]:
from aves.visualization.networks import NodeLink


In [ ]:
from aves.visualization.figures import figure_from_geodataframe



In [ ]:
import pyrosm

# redes con OSM
OSM_PATH = AVES_ROOT / "data" / "external" / "OSM"
osm = pyrosm.OSM(str(OSM_PATH / 'clipped-scl-osm.pbf'))

In [ ]:
network_nodes, network_edges  = osm.get_network(nodes=True, network_type='driving')

In [ ]:
network_nodes

In [ ]:
network_edges['highway'].value_counts()

In [ ]:
network_edges[network_edges['highway'].isin(['motorway', 'primary', 'secondary'])]

In [ ]:
network_edges[['u', 'v', 'length']]

In [ ]:
osm_network = Network.from_edgelist(
    network_edges[network_edges['highway'].isin(['motorway', 'primary', 'secondary'])],
    source="u",
    target="v",
    weight="length",
)

In [ ]:
connected_network = osm_network.largest_connected_component(directed=False)

In [ ]:
osm_network.num_vertices, connected_network.num_vertices

In [ ]:
osm_network.num_edges, connected_network.num_edges

In [ ]:
nodelink = NodeLink(connected_network)
nodelink.layout_nodes(
    method="geographical", geodataframe=network_nodes, node_column="id"
)


In [ ]:
fig, ax = figure_from_geodataframe(zones, height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=1, alpha=1.0),
    edges=dict(alpha=0.25, linewidth=0.25),
    zorder=2,
)

fig.tight_layout()


In [ ]:
pagerank = connected_network.estimate_pagerank()
pagerank

In [ ]:
pd.Series(pagerank.a).plot(kind='hist', bins=100)

In [ ]:
nodelink.set_node_drawing('plain', weights=np.power(np.array(pagerank.a), 2))

In [ ]:
fig, ax = figure_from_geodataframe(zones, height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=50, alpha=0.01),
    edges=dict(alpha=0.1, linewidth=0.25),
    zorder=2,
)

fig.tight_layout()

In [ ]:
node_betweenness, edge_betweenness = connected_network.estimate_betweenness()

In [ ]:
pd.Series(node_betweenness.a).plot(kind='hist', bins=100)

In [ ]:
pd.Series(edge_betweenness.a).plot(kind='hist', bins=100)

In [ ]:
nodelink.set_node_drawing('plain', weights=np.power(np.array(node_betweenness.a), 2))

In [ ]:
nodelink.set_edge_drawing('weighted', weights=np.sqrt(np.array(edge_betweenness.a)))

In [ ]:
fig, ax = figure_from_geodataframe(zones, height=9)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=10, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=3, palette='YlOrRd', ),
    zorder=2,
)

fig.tight_layout()

In [ ]:
# camino más corto
src_point = (-70.6651616, -33.4582698)
dst_point = (-70.6397468, -33.4391656)


In [ ]:
import shapely.geometry

_network_nodes = network_nodes[network_nodes['id'].isin(connected_network.node_map.keys())]

def get_node_id(query_point):
    query_point_node_id = _network_nodes.distance(
        shapely.geometry.Point(query_point)
    ).idxmin()
    return _network_nodes.loc[query_point_node_id]['id']


In [ ]:
src_node_id = get_node_id(src_point)
dst_node_id = get_node_id(dst_point)

src_node_id, dst_node_id

In [ ]:
path = connected_network.shortest_path(src_node_id, dst_node_id)
len(path)

In [ ]:
len(path[0])

In [ ]:
path_network = connected_network.subgraph(nodes=path[0])
path_nodelink = NodeLink(path_network)
path_nodelink.layout_nodes(
    method="geographical", geodataframe=network_nodes, node_column="id"
)

In [ ]:
fig, ax = figure_from_geodataframe(zones[zones["NOM_COMUNA"] == "SANTIAGO"], height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=10, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=2, palette='Greys', ),
    zorder=2,
)

path_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=1, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=5, color='red'),
    zorder=2,
)

fig.tight_layout()


In [ ]:
import graph_tool.topology

min_tree = graph_tool.topology.min_spanning_tree(connected_network.graph, weights=connected_network._edge_weight)
min_tree

In [ ]:
min_tree_network = connected_network.subgraph(edge_filter=min_tree)
min_tree_network.graph

In [ ]:
tree_nodelink = NodeLink(min_tree_network)
tree_nodelink.layout_nodes(
    method="geographical", geodataframe=network_nodes, node_column="id"
)

In [ ]:
fig, ax = figure_from_geodataframe(zones[zones["NOM_COMUNA"] == "SANTIAGO"], height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=10, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=2, palette='Greys', ),
    zorder=2,
)

tree_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=1, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=5, color='red'),
    zorder=2,
)

fig.tight_layout()

In [ ]:
bike_nodes, bike_edges  = osm.get_network(nodes=True, network_type='cycling')

In [ ]:
bike_cycleways = bike_edges.pipe(lambda x: x[pd.notnull(x["tags"])]).pipe(
    lambda x: x[x["tags"].str.contains("cycleway|bicycle")]
)
bike_cycleways


In [ ]:
bike_edges['cycleway'].value_counts()

In [ ]:
bike_lanes = bike_edges[pd.notnull(bike_edges['cycleway'])]
bike_lanes.plot()

In [ ]:
fig, ax = figure_from_geodataframe(zones[zones["NOM_COMUNA"].isin(["SANTIAGO", "PROVIDENCIA", "ÑUÑOA", 'LAS CONDES', 'LA FLORIDA'])], height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)
bike_lanes.plot(ax=ax)
bike_cycleways.plot(ax=ax)

fig.tight_layout()

In [ ]:
bike_relevant_nodes = set(bike_cycleways['u']) | set(bike_lanes['u']) | set(bike_cycleways['v']) | set(bike_lanes['v'])
len(bike_relevant_nodes)

In [ ]:
bike_network = Network.from_edgelist(
    bike_edges[bike_edges['u'].isin(bike_relevant_nodes) & bike_edges['v'].isin(bike_relevant_nodes)],
    source="u",
    target="v",
    weight="length",
)

In [ ]:
bike_components = bike_network.connected_components(directed=False)
bike_components

In [ ]:
fig, ax = figure_from_geodataframe(
    zones[
        zones["NOM_COMUNA"].isin(
            ["SANTIAGO", "PROVIDENCIA", "ÑUÑOA", "LAS CONDES", "LA FLORIDA"]
        )
    ],
    height=7,
)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

bike_edges.join(
    pd.Series(
        bike_components[0].a, index=bike_network.node_map.keys(), name="component"
    ),
    on="u",
).plot(column="component", ax=ax)

fig.tight_layout()


In [ ]:
bike_largest_component = bike_network.largest_connected_component(directed=False)
bike_lcc_nodelink = NodeLink(bike_largest_component)
bike_lcc_nodelink.layout_nodes(
    method="geographical", geodataframe=bike_nodes, node_column="id"
)

In [ ]:
fig, ax = figure_from_geodataframe(
    zones[
        zones["NOM_COMUNA"].isin(
            ["SANTIAGO", "PROVIDENCIA", "ÑUÑOA", "LAS CONDES", "LA FLORIDA"]
        )
    ],
    height=7,
)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

bike_lcc_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=1, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=2),
    zorder=2,
)


fig.tight_layout()